In [24]:
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
import os
import glob
import shutil
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import random

### Gensim pretrain MODEL

In [25]:
import gensim.downloader
from gensim.models import KeyedVectors

In [26]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-200') 
#glove-wiki-gigaword-200   #'glove-twitter-200'
#glove-wiki-gigaword-100   #glove-twitter-100'
#word2vec-google-news-300  #fasttext-wiki-news-subwords-300  #'glove-wiki-gigaword-300'
#glove-wiki-gigaword-50    #glove-twitter-50

## Setting

In [43]:
path='./AWA2_33c/test/'

'''下三行:吃圖片的路徑'''    
train_path=Path("./data/AWA2/IMG_backoff/train")
test_path=Path("./data/AWA2/IMG_backoff/test")
val_path=Path("./data/AWA2/IMG_backoff/val")

## DataSet

In [28]:
train_classes = []
train_images = []
train_img_vecs = []
test_classes=[]
test_images = []
test_img_vecs = []
train_class_labels=[]
val_class_labels=[]
test_class_labels=[]

In [29]:
val_classes = []
val_images = []
val_img_vecs = []

### Train Data

In [30]:
#label轉Word2Vector
label_c=0
for class_name in train_path.iterdir():
    c = class_name.name.rpartition(".")[-1]
    print(c)
    if c in glove_vectors:
        class_images = list(class_name.iterdir())
        train_classes.append(c)
        train_images += class_images
        train_img_vecs += [glove_vectors[c]]*len(class_images)
        for k in class_name.iterdir():  
            train_class_labels.append(label_c)
        label_c+=1

antelope
bat
beaver
blue_whale
bobcat
buffalo
chihuahua
collie
cow
dalmatian
deer
dolphin
elephant
fox
german_shepherd
giraffe
gorilla
grizzly_bear
hamster
horse
killer_whale
lion
mole
moose
mouse
otter
ox
polar_bear
rabbit
rhinoceros
sheep
siamese_cat
skunk
spider_monkey
squirrel
tiger
walrus
weasel
wolf
zebra


#### Check

In [31]:
print(len(train_classes))
print(len(train_images))
print(len(train_img_vecs))
print(len(train_class_labels))
print((train_class_labels))

33
21057
21057
21057
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [32]:
train_img_vecs = np.stack(train_img_vecs)
print(train_img_vecs.shape)

(21057, 200)


In [33]:
#資料打亂打平均用的，但這樣label可能會亂掉，Training可以用。
grouped = list(zip(train_images, train_img_vecs,train_class_labels)) 
random.shuffle(grouped) 
train_images, train_img_vecs,train_class_labels= zip(*grouped)

### Val_Data

In [34]:
label_c=0
for class_name in val_path.iterdir():
    c = class_name.name.rpartition(".")[-1]
    if c in glove_vectors:
        class_images = list(class_name.iterdir())
        val_classes.append(c)
        val_images += class_images
        val_img_vecs += [glove_vectors[c]]*len(class_images)
        for k in class_name.iterdir():  
            val_class_labels.append(label_c)
        label_c+=1

#### Check

In [35]:
print(len(val_classes))
print(len(val_images))
print(len(val_img_vecs))
print(val_classes)
#print(train_classes)
print(len(val_class_labels))

33
5268
5268
['antelope', 'bat', 'beaver', 'bobcat', 'buffalo', 'chihuahua', 'collie', 'cow', 'dalmatian', 'deer', 'dolphin', 'elephant', 'fox', 'giraffe', 'gorilla', 'hamster', 'horse', 'lion', 'mole', 'moose', 'mouse', 'otter', 'ox', 'rabbit', 'rhinoceros', 'sheep', 'skunk', 'squirrel', 'tiger', 'walrus', 'weasel', 'wolf', 'zebra']
5268


In [36]:
val_img_vecs  = np.stack(val_img_vecs)
print(val_img_vecs.shape)

(5268, 200)


In [37]:
grouped_val = list(zip(val_images, val_img_vecs,val_class_labels)) 
random.shuffle(grouped_val) 
val_images, val_img_vecs,val_class_labels = zip(*grouped_val)

### Test Data

In [38]:
label_c=0
for class_name in test_path.iterdir():
    c = class_name.name.rpartition(".")[-1]
    if c in glove_vectors:
        class_images = list(class_name.iterdir())
        test_classes.append(c)
        test_images += class_images
        test_img_vecs += [glove_vectors[c]]*len(class_images)
        for k in class_name.iterdir():  
            test_class_labels.append(label_c)
        label_c+=1

#### Check

In [39]:
print(len(test_classes))
print(len(test_images))
print(len(test_img_vecs))
print(test_classes)
print(len(test_class_labels))

7
4654
4654
['chimpanzee', 'hippopotamus', 'leopard', 'pig', 'raccoon', 'rat', 'seal']
4654


In [40]:
test_img_vecs  = np.stack(test_img_vecs)
print(test_img_vecs.shape)

(4654, 200)


### Data_Save

In [41]:
np.save(path+'train_img_name.npy', train_images)
np.save(path+'train_img_vec.npy', train_img_vecs)
np.save(path+'train_class.npy', train_classes)
np.save(path+'val_img_name.npy', val_images)
np.save(path+'val_img_vec.npy', val_img_vecs)
np.save(path+'val_img_class.npy', val_classes)
np.save(path+'test_img_name.npy', test_images)
np.save(path+'test_img_vec.npy', test_img_vecs)
np.save(path+'test_img_class.npy', test_classes)
np.save(path+'test_img_label.npy', test_class_labels)
np.save(path+'val_img_label.npy', val_class_labels)
np.save(path+'train_img_label.npy', train_class_labels)

## Val+test=MIX

In [42]:
Classes=[]
Images=[]
Img_vec=[]
Class_labels=[]
label_c=0
for class_name in val_path.iterdir():
    c = class_name.name.rpartition(".")[-1]
    if c in glove_vectors:
        class_images = list(class_name.iterdir())
        Classes.append(c)
        Images += class_images
        Img_vec += [glove_vectors[c]]*len(class_images)
        for k in class_name.iterdir():  
            Class_labels.append(label_c)
        label_c+=1
for class_name in test_path.iterdir():
    c = class_name.name.rpartition(".")[-1]
    if c in glove_vectors:
        class_images = list(class_name.iterdir())
        Classes.append(c)
        Images += class_images
        Img_vec += [glove_vectors[c]]*len(class_images)
        for k in class_name.iterdir():  
            Class_labels.append(label_c)
        label_c+=1
Img_vec  = np.stack(Img_vec)
print(Img_vec.shape)
np.save(path+'mix_img_name.npy', Images)
np.save(path+'mix_img_vec.npy', Img_vec)
np.save(path+'mix_img_class.npy', Classes)
np.save(path+'mix_img_label.npy', Class_labels)

(9922, 200)


In [44]:
del glove_vectors